# NELA Article Collection

Created: 2019.10.8  
Notebook sequence: 5

For getting code together to build the sets of articles to use for each indicator set

---

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import util
import pandas as pd
from tqdm import tqdm_notebook as tqdm

In [20]:
def random_balanced_sample(source_name_array, count, reject_minimum=100, force_balance=True, random_seed=13, verbose=True):
    building_df = None
    counts = {}
    minimum_count = 100000000 # derp
    returned_counts = {}
    rejected = []
    
    for name in tqdm(source_name_array, "Querying sources", disable=(not verbose)):
        local_df = util.nela_load_articles_from_source(name)
        local_count = local_df.shape[0]
        if verbose: print(name, local_count, "articles")
        
        if local_count < reject_minimum:
            rejected.append(name)
            if verbose: print(name, "rejected")
            continue
    
        if local_count < minimum_count:
            minimum_count = local_count
    
        counts[name] = local_count
        building_df = util.stack_dfs(building_df, local_df)
        
    max_possible_balanced = minimum_count*len(counts.keys())
    if verbose: print(max_possible_balanced,"maximum possible balanced sample size")
        
    sample_df = None
    if count > max_possible_balanced:
        if verbose: print("Grabbing maximum")
        if force_balance:
            if verbose: print("Force balance requested, will not return", count, "as requested")
            
            for name in tqdm(counts.keys(), "Sampling", disable=(not verbose)):
                source_sample_df = building_df[building_df.source == name].sample(minimum_count, random_state=random_seed)
                sample_df = util.stack_dfs(sample_df, source_sample_df)
        
        else:
            print("WARNING: unbalanced output")
            
            # TODO
    else:
        total_per = int(count / len(counts.keys()))
        remainder = count % len(counts.keys())
        if verbose: print("Grabbing", total_per, "per source")
        
        for name in tqdm(counts.keys(), "Sampling", disable=(not verbose)):
            sample_size = total_per
            if remainder > 0: 
                sample_size += 1
                remainder -= 1
            source_sample_df = building_df[building_df.source == name].sample(sample_size, random_state=random_seed)
            returned_counts[name] = source_sample_df.shape[0]
            sample_df = util.stack_dfs(sample_df, source_sample_df)

                
    return sample_df, returned_counts, rejected
    

In [12]:
# see notebook 2
os_unreliable = ['Addicting Info', 'Breitbart', 'CNS News', 'Intellihub', 'LewRockwell', 'NODISINFO', 'Politicus USA', 'Shareblue', 'The Duran', 'The Gateway Pundit', 'The Political Insider', 'The Washington Examiner', 'TheAntiMedia', 'True Activist', 'Veterans Today']

In [18]:
df, counts, rejected = random_balanced_sample(os_unreliable, 4500)

Addicting Info 429 articles
Breitbart 1877 articles
CNS News 5263 articles
Intellihub 334 articles
LewRockwell 1278 articles
NODISINFO 29 articles
NODISINFO rejected
Politicus USA 4018 articles
Shareblue 2134 articles
The Duran 959 articles
The Gateway Pundit 5667 articles
The Political Insider 2680 articles
The Washington Examiner 469 articles
TheAntiMedia 666 articles
True Activist 370 articles
Veterans Today 2624 articles

4676 maximum possible balanced sample size
Grabbing 321 per source


In [19]:
counts

{'Addicting Info': 322,
 'Breitbart': 322,
 'CNS News': 322,
 'Intellihub': 322,
 'LewRockwell': 322,
 'Politicus USA': 322,
 'Shareblue': 321,
 'The Duran': 321,
 'The Gateway Pundit': 321,
 'The Political Insider': 321,
 'The Washington Examiner': 321,
 'TheAntiMedia': 321,
 'True Activist': 321,
 'Veterans Today': 321}